In [ ]:
# ================================================== [ setting ] ==================================================
# !pip install konlpy
import pandas as pd 
import re
from konlpy.tag import Komoran
from gensim import corpora
import gensim
import pyLDAvis.gensim
import warnings
warnings.filterwarnings(action = 'ignore')

news_data = pd.read_csv("kyungnam_news_total.csv", encoding = "utf-8")

In [ ]:
# ============================================== [ LDA modeling ] ======================================================
def LDA_modeling(df) :
  df = df.str.replace("\([^)]*\)", "") # 괄호안의 내용 제거 ex) 코로나(COVID-19) -> 코로나
  df = df.str.replace("\n\n", "") # 문단 구분자 제거 
  df = df.str.replace("[^a-zA-Z가-힣 ]", "") # 숫자 / 영어 / 한글 제외하고 모두 제거 

  processed_data = []
  for sentence in df : 
    tag = komoran.pos(sentence)
    nouns = [s for s, t in tag if t in ['SL', 'NNG', 'NNP'] and len(s) > 1] # 명사 태그 추출, 한글자 이상만 추출
    processed_data.append(nouns) # token 형태로 list에 저장

  dictionary = corpora.Dictionary(processed_data)
  corpus = [dictionary.doc2bow(text) for text in processed_data]

  NUM_TOPICS = 3
  ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes = 15)
  topics = ldamodel.print_topics(num_words = 10)
  
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
  
  return pyLDAvis.display(vis)

In [ ]:
LDA_modeling(news_data["new_summary"])